In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical 
import numpy as np
from keras.preprocessing import image

# Plant

In [2]:
specific_train = 'corn'

# Definitions

In [3]:
image_size = tuple((64, 64))
height = image_size[0]
width = image_size[1]
depth = 3

# Part 1 - Data Preprocessing

### Preprocessing the Training set

In [4]:

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/train_images/' + specific_train,
                                                 target_size = image_size,
                                                 batch_size = 32)

Found 3927 images belonging to 4 classes.


In [5]:
num_classes = len(training_set.class_indices)
train_labels = training_set.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)
class_indices = training_set.class_indices
class_indices

{'blight': 0, 'gray': 1, 'healthly': 2, 'rust': 3}

### Preprocessing the Test set

In [6]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_images/' + specific_train,
                                            target_size = image_size,
                                            batch_size = 32)

Found 1120 images belonging to 4 classes.


# Part 2 - Building the CNN


In [7]:
cnn = tf.keras.models.Sequential()

Step 1 - Convolution


In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(height, width, depth)))

Step 2 - Pooling


In [9]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

Adding a second convolutional layer

In [10]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

Step 3 - Flattening

In [11]:
cnn.add(tf.keras.layers.Flatten())

Step 4 - Full Connection

In [12]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

Step 5 - Output Layer

In [13]:
cnn.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

# Part 3 - Training the CNN

Compiling the CNN

In [14]:
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Training the CNN on the Training set and evaluating it on the Test set

In [15]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 15)

Epoch 1/15
123/123 [==============================] - 18s 144ms/step - loss: 0.5525 - accuracy: 0.7830 - val_loss: 0.4546 - val_accuracy: 0.8170
Epoch 2/15
123/123 [==============================] - 18s 146ms/step - loss: 0.3256 - accuracy: 0.8569 - val_loss: 0.4222 - val_accuracy: 0.8107
Epoch 3/15
123/123 [==============================] - 18s 147ms/step - loss: 0.3153 - accuracy: 0.8671 - val_loss: 0.5087 - val_accuracy: 0.7973
Epoch 4/15
123/123 [==============================] - 18s 143ms/step - loss: 0.2749 - accuracy: 0.8826 - val_loss: 0.4211 - val_accuracy: 0.8295
Epoch 5/15
123/123 [==============================] - 17s 142ms/step - loss: 0.2503 - accuracy: 0.8958 - val_loss: 0.3567 - val_accuracy: 0.8446
Epoch 6/15
123/123 [==============================] - 18s 145ms/step - loss: 0.2464 - accuracy: 0.8930 - val_loss: 0.3520 - val_accuracy: 0.8589
Epoch 7/15
123/123 [==============================] - 18s 147ms/step - loss: 0.2383 - accuracy: 0.9058 - val_loss: 0.3504 - val_ac

In [16]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        4128      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 7200)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               921728    
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 5

# Making a prediction

In [44]:
def read_image(image_name):
    test_image = image.load_img(f'dataset/single_prediction/{specific_train}/{prediction_image}', target_size = image_size)
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    test_image /= 255
    return test_image

In [45]:
def test_single_image(prediction_image):
    images = read_image(prediction_image)
    result = cnn.predict(images)
    return result

In [46]:
def print_prediction(result):
    result_indice = np.argmax(result[0])
    prediction = list(class_indices.keys())[result_indice]
    prediction_percentage = max(result[0]) * 100
    print(f"Classes = {class_indices}")
    print("Prediction = {} - {:.2f}%".format(prediction, prediction_percentage))

In [51]:
prediction_image = 'blight.jpeg'
result = test_single_image(prediction_image)
print(result[0])
print_prediction(result)

[5.1721287e-01 4.0081370e-01 5.4597411e-05 8.1918851e-02]
Classes = {'blight': 0, 'gray': 1, 'healthly': 2, 'rust': 3}
Prediction = blight - 51.72%


# Saving the model

In [24]:
cnn.save(f'model/{specific_train}')

INFO:tensorflow:Assets written to: model/corn/assets
